In [67]:
import pulp

In [ ]:
print("hi")

In [68]:
n = 10
soc = [50,50,50,50,50,50,50,50,50,50]
stations = [20,30,50]
total_distance = sum(stations)
speed = 10
charging_rate = 10
arrival_time = [0,2,4,6,8,10,12,14,16,18]
T = 1000

In [69]:
#initiate the model
cs = pulp.LpProblem("station",pulp.LpMinimize)
vehicles=[]
#add variables
soc_vehicle = {}
arrival_time_vehicle = {}
for i in range(n):
    vehicles.append(f"vehicle{i}")
    soc_vehicle[f"vehicle{i}"] = soc[i]
    arrival_time_vehicle[f"vehicle{i}"] = arrival_time[i]
M = 10000
print(vehicles)
#arrival time at stations
a1 = pulp.LpVariable.dict("Arrival Time 1",[vehicle for vehicle in vehicles],lowBound=0,upBound=T,cat=pulp.LpInteger)
a2 = pulp.LpVariable.dict("Arrival Time 2",[vehicle for vehicle in vehicles],lowBound=0,upBound=T,cat=pulp.LpInteger)
# servicing time at stations
s1 = pulp.LpVariable.dict("Service Time 1",[vehicle for vehicle in vehicles],lowBound=0,cat=pulp.LpInteger)
s2 = pulp.LpVariable.dict("Service Time 2",[vehicle for vehicle in vehicles],lowBound=0,cat=pulp.LpInteger)
#initial charging time at sations
u1 = pulp.LpVariable.dict("initial charge time 1",[vehicle for vehicle in vehicles],lowBound=0,cat=pulp.LpInteger)
u2 = pulp.LpVariable.dict("initial charge time 2",[vehicle for vehicle in vehicles],lowBound=0,cat=pulp.LpInteger)
#final charge time at stations
d1 = pulp.LpVariable.dict("final charge time 1",[vehicle for vehicle in vehicles],lowBound=0,cat=pulp.LpInteger)
d2 = pulp.LpVariable.dict("final charge time 2",[vehicle for vehicle in vehicles],lowBound=0,cat=pulp.LpInteger)
# sigma- temporal ordering of vehicles
o1 = pulp.LpVariable.dict("sigma 1",[vehicle1+vehicle2 for vehicle1 in vehicles for vehicle2 in vehicles if vehicle1 != vehicle2],lowBound=0,cat=pulp.LpBinary)
o2 = pulp.LpVariable.dict("sigma 2",[vehicle1+vehicle2 for vehicle1 in vehicles for vehicle2 in vehicles if vehicle1 != vehicle2],lowBound=0,cat=pulp.LpBinary)
#



['vehicle0', 'vehicle1', 'vehicle2', 'vehicle3', 'vehicle4', 'vehicle5', 'vehicle6', 'vehicle7', 'vehicle8', 'vehicle9']


In [70]:
#objective function
cs += pulp.lpSum([d1[vehicle]-a1[vehicle] + d2[vehicle]-a2[vehicle]] for vehicle in vehicles)
#blending+= (pulp.lpSum([4.32*ing_weight[(i,"pork")]+2.46*ing_weight[(i,"wheat")]+1.86*ing_weight[(i,"starch")] for i in sausages]))

In [77]:
#constraints
for vehicle in vehicles:
    #departure = service + charge start
    cs += d1[vehicle] == s1[vehicle]+u1[vehicle]
    cs += d2[vehicle] == s2[vehicle]+u2[vehicle]
    
    # charge start > arival time
    cs += u1[vehicle]>=a1[vehicle]
    cs += u2[vehicle]>=a2[vehicle]
    cs += u2[vehicle]<=T-s2[vehicle]
    
    # arrival times of each station
    cs += a1[vehicle] == arrival_time_vehicle[vehicle]+(stations[0]/speed)
    cs += a2[vehicle] == (stations[1]/speed) + d1[vehicle]
    
    # ensuring soc capabilities
    cs += charging_rate*s1[vehicle] + soc_vehicle[vehicle]>=stations[0]+stations[1]
    cs += charging_rate*s1[vehicle] + soc_vehicle[vehicle]<=100
    cs += charging_rate*(s2[vehicle]+s1[vehicle])+soc_vehicle[vehicle]>=stations[0]+stations[1]+stations[2]
    cs += charging_rate*(s2[vehicle]+s1[vehicle])+soc_vehicle[vehicle]-stations[0]-stations[1]<=100
for vehicle1 in vehicles:
    
    for vehicle2 in vehicles:
        if vehicle1!=vehicle2:
            #making sure o1[vehicle1+vehicle2] = 1 when vehicle1 in front of vehicle2
            cs += M*o1[vehicle1+vehicle2] >= a1[vehicle2]-a1[vehicle1]
            cs += M*(1-o1[vehicle1+vehicle2])>= a1[vehicle1]-a1[vehicle2]
            cs += M*o2[vehicle1+vehicle2] >= a2[vehicle2]-a2[vehicle1]
            cs += M*(1-o2[vehicle1+vehicle2]) >= a2[vehicle1]-a2[vehicle2]
            #implementing wait time
            cs += u1[vehicle2]-u1[vehicle1]-s1[vehicle1]-(o1[vehicle1+vehicle2]-1)*T>= 0
            cs += u2[vehicle2]-u2[vehicle1]-s2[vehicle1]-(o2[vehicle1+vehicle2]-1)*T>= 0
            #making sure only one vehicle is front of another
            cs += o1[vehicle1+vehicle2] + o1[vehicle2+vehicle1] >= 1
            cs += o2[vehicle1+vehicle2] + o2[vehicle2+vehicle1] >= 1




TypeError: Non-constant expressions cannot be multiplied

1727.33 seconds)
Cbc0010I After 433000 nodes, 85108 on tree, 137 best solution, best possible 106 (1729.22 seconds)
Cbc0010I After 434000 nodes, 85450 on tree, 137 best solution, best possible 106.004 (1733.71 seconds)
Cbc0010I After 435000 nodes, 85659 on tree, 137 best solution, best possible 106.004 (1736.68 seconds)
Cbc0010I After 436000 nodes, 85841 on tree, 137 best solution, best possible 106.004 (1739.88 seconds)
Cbc0010I After 437000 nodes, 85747 on tree, 137 best solution, best possible 106.004 (1742.17 seconds)
Cbc0010I After 438000 nodes, 86023 on tree, 137 best solution, best possible 106.01885 (1746.51 seconds)
Cbc0010I After 439000 nodes, 86259 on tree, 137 best solution, best possible 106.01885 (1749.93 seconds)
Cbc0010I After 440000 nodes, 86441 on tree, 137 best solution, best possible 106.01885 (1753.60 seconds)
Cbc0010I After 441000 nodes, 86394 on tree, 137 best solution, best possible 106.01885 (1755.34 seconds)
Cbc0010I After 442000 nodes, 86670 on tree, 137 best

In [72]:
cs

station:
MINIMIZE
-1*Arrival_Time_1_vehicle0 + -1*Arrival_Time_1_vehicle1 + -1*Arrival_Time_1_vehicle2 + -1*Arrival_Time_1_vehicle3 + -1*Arrival_Time_1_vehicle4 + -1*Arrival_Time_1_vehicle5 + -1*Arrival_Time_1_vehicle6 + -1*Arrival_Time_1_vehicle7 + -1*Arrival_Time_1_vehicle8 + -1*Arrival_Time_1_vehicle9 + -1*Arrival_Time_2_vehicle0 + -1*Arrival_Time_2_vehicle1 + -1*Arrival_Time_2_vehicle2 + -1*Arrival_Time_2_vehicle3 + -1*Arrival_Time_2_vehicle4 + -1*Arrival_Time_2_vehicle5 + -1*Arrival_Time_2_vehicle6 + -1*Arrival_Time_2_vehicle7 + -1*Arrival_Time_2_vehicle8 + -1*Arrival_Time_2_vehicle9 + 1*final_charge_time_1_vehicle0 + 1*final_charge_time_1_vehicle1 + 1*final_charge_time_1_vehicle2 + 1*final_charge_time_1_vehicle3 + 1*final_charge_time_1_vehicle4 + 1*final_charge_time_1_vehicle5 + 1*final_charge_time_1_vehicle6 + 1*final_charge_time_1_vehicle7 + 1*final_charge_time_1_vehicle8 + 1*final_charge_time_1_vehicle9 + 1*final_charge_time_2_vehicle0 + 1*final_charge_time_2_vehicle1 + 1*fina

In [73]:
cs.solve()

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/sushi/Desktop/internship/Routing/.venv/lib/python3.8/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/9c8aa0ec401b463b89982a15d536c3e1-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /tmp/9c8aa0ec401b463b89982a15d536c3e1-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 835 COLUMNS
At line 3766 RHS
At line 4597 BOUNDS
At line 4858 ENDATA
Problem MODEL has 830 rows, 260 columns and 2370 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 50 - 0.00 seconds
Cgl0003I 0 fixed, 10 tightened bounds, 12 strengthened rows, 0 substitutions
Cgl0004I processed model has 400 rows, 140 columns (140 integer (90 of which binary)) and 1215 elements
Cutoff increment increased from 1e-05 to 0.9999
Cbc0045I 20 integer variables out of 140 objects (140 integer) have cost of 1 - high priority

1

In [74]:
pulp.LpStatus[cs.status]

'Optimal'

In [75]:
for vehicle in vehicles:
    print(f"s1: {s1[vehicle].varValue}, s2: {s2[vehicle].varValue}, a1,a2 : {a1[vehicle].varValue},{a2[vehicle].varValue}, u1,u2 :{u1[vehicle].varValue}.{u2[vehicle].varValue} d1,d2 :{d1[vehicle].varValue}.{d2[vehicle].varValue}")

s1: 2.0, s2: 3.0, a1,a2 : 2.0,7.0, u1,u2 :2.0.7.0 d1,d2 :4.0.10.0
s1: 3.0, s2: 2.0, a1,a2 : 4.0,10.0, u1,u2 :4.0.10.0 d1,d2 :7.0.12.0
s1: 2.0, s2: 3.0, a1,a2 : 6.0,12.0, u1,u2 :7.0.12.0 d1,d2 :9.0.15.0
s1: 3.0, s2: 2.0, a1,a2 : 8.0,15.0, u1,u2 :9.0.15.0 d1,d2 :12.0.17.0
s1: 2.0, s2: 3.0, a1,a2 : 10.0,17.0, u1,u2 :12.0.17.0 d1,d2 :14.0.20.0
s1: 3.0, s2: 2.0, a1,a2 : 12.0,20.0, u1,u2 :14.0.20.0 d1,d2 :17.0.22.0
s1: 2.0, s2: 3.0, a1,a2 : 14.0,22.0, u1,u2 :17.0.22.0 d1,d2 :19.0.25.0
s1: 3.0, s2: 2.0, a1,a2 : 16.0,25.0, u1,u2 :19.0.25.0 d1,d2 :22.0.27.0
s1: 2.0, s2: 3.0, a1,a2 : 18.0,27.0, u1,u2 :22.0.27.0 d1,d2 :24.0.30.0
s1: 3.0, s2: 2.0, a1,a2 : 20.0,30.0, u1,u2 :24.0.30.0 d1,d2 :27.0.32.0


In [76]:
print(pulp.value(cs.objective))

70.0


e, 137 best solution, best possible 105.09465 (1579.21 seconds)
Cbc0010I After 394000 nodes, 78388 on tree, 137 best solution, best possible 105.21155 (1585.91 seconds)
Cbc0010I After 395000 nodes, 78660 on tree, 137 best solution, best possible 105.21155 (1590.15 seconds)
Cbc0010I After 396000 nodes, 78811 on tree, 137 best solution, best possible 105.21155 (1593.38 seconds)
Cbc0010I After 397000 nodes, 78738 on tree, 137 best solution, best possible 105.21155 (1595.20 seconds)
Cbc0010I After 398000 nodes, 79078 on tree, 137 best solution, best possible 105.32529 (1601.56 seconds)
Cbc0010I After 399000 nodes, 79277 on tree, 137 best solution, best possible 105.32529 (1605.22 seconds)
Cbc0010I After 400000 nodes, 79504 on tree, 137 best solution, best possible 105.32529 (1608.62 seconds)
Cbc0010I After 401000 nodes, 79416 on tree, 137 best solution, best possible 105.32529 (1610.47 seconds)
Cbc0010I After 402000 nodes, 79755 on tree, 137 best solution, best possible 105.45149 (1616.90 